In [1]:
%reload_ext autoreload
%autoreload 2

from sklearn.model_selection import KFold
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from preprocessing_train_test import *

from scipy.stats import skew 
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

pd.set_option('display.max_columns', 100)  # 设置显示100列
pd.set_option('display.max_rows', 100)   # 设置显示100行

/Users/hudongyue/opt/anaconda3/envs/envname/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
file_path_train = "../data/backup/merge_auxiliary_data_train.csv"
file_path_test = "../data/backup/merge_auxiliary_data_test.csv"

train_gdf = pd.read_csv(file_path_train)
test_gdf = pd.read_csv(file_path_test)

print("train dataset shape: ", train_gdf.shape)
print("test dataset shape: ",test_gdf.shape)

train dataset shape:  (431732, 55)
test dataset shape:  (107934, 54)


## XGBRegressor

In [46]:
max_depths = [6, 7, 8, 9]
min_child_weights = [1.2, 1.3, 1.4]
reg_alphas = [1.5, 1.6, 1.7] 
reg_lambdas = [1.5, 1.55, 1.6]

scores_xgb = {}
for max_depth in max_depths:
    for min_child_weight in min_child_weights:
        for reg_alpha in reg_alphas:
            for reg_lambda in reg_lambdas:
                print('start training with max_depth: {}, min_child_weight: {}, reg_alpha: {}, reg_lambda: {}'.format(max_depth, min_child_weight, reg_alpha, reg_lambda))
                mae_test, mae_valid, mse_test, mse_valid = [], [], [], []
                split = KFold(n_splits=5, shuffle=True).split(train_gdf)
                for idx, (train_idx, validate_idx) in enumerate(split):
                    print('fold {}'.format(idx + 1))
                    train_df, validate_df = preprocess_train_test(train_gdf.iloc[list(train_idx)], train_gdf.iloc[list(validate_idx)])

                    x_train, y_train = train_df.drop(columns=["resale_price"], errors='ignore'), train_df["resale_price"]
                    x_valid, y_valid = validate_df.drop(columns=["resale_price"], errors='ignore'), validate_df["resale_price"]

                    features = pd.concat([x_train, x_valid]).reset_index(drop=True)
                    # overfit = []
                    # for i in features.columns:
                    #     counts = features[i].value_counts()
                    #     zeros = counts.iloc[0]
                    #     if zeros / len(features) * 100 > 99.94:
                    #         overfit.append(i)

                    # overfit = list(overfit)
                    final_features = features.drop(columns=['primary_id', 'second_id', 'mall_id'], axis=1)
                    # final_features = features.drop(columns=overfit, axis=1)
                    # final_features = features.iloc[:, :60]

                    x_train = final_features.iloc[:len(x_train), :]
                    x_valid = final_features.iloc[len(x_train):, :]

                    regressor = xgb.XGBRegressor(n_estimators=1400, max_depth=max_depth, min_child_weight=min_child_weight, reg_alpha=reg_alpha, reg_lambda=reg_lambda, subsample=1, n_jobs=3).fit(x_train, y_train)
                    
                    y_train_predict = regressor.predict(x_train)
                    y_valid_predict = regressor.predict(x_valid)

                    mae_test.append(mean_absolute_error(y_train, y_train_predict))
                    mae_valid.append(mean_absolute_error(y_valid, y_valid_predict))
                    mse_test.append(mean_squared_error(y_train, y_train_predict))
                    mse_valid.append(mean_squared_error(y_valid, y_valid_predict))

                print(mae_test, mae_valid, mse_test, mse_valid)
                scores_xgb[(max_depth, min_child_weight, reg_alpha, reg_lambda)] = (mae_test, mae_valid, mse_test, mse_valid)

start training with max_depth: 6, min_child_weight: 1.3, reg_alpha: 1.6, reg_lambda: 1.55
fold 1
fold 2
fold 3
fold 4
fold 5
[9266.453587621168, 9287.203001335938, 9282.786987583408, 9308.06973242891, 9289.382392380516] [11721.26806127095, 11803.310206882186, 11792.703757158697, 11779.721233817287, 11766.766135597987] [157818613.32877317, 158714033.60133395, 158330645.71937108, 159237465.06544283, 158469388.71588218] [278139624.66029316, 279817605.14267087, 280596078.13053757, 281832190.6555247, 282603805.1240578]


In [47]:
min_mae_key, min_mae = None, 100000
min_mse_key, min_mse = None, 10000000000000
for key, (_, mae_valid, _, mse_valid) in scores_xgb.items():
    if np.mean(mae_valid) < min_mae:
        min_mae_key = key
        min_mae = np.mean(mae_valid)
    if np.mean(mse_valid) < min_mse:
        min_mse_key = key
        min_mse = np.mean(mse_valid)
print(min_mae_key, min_mae)
print(min_mse_key, min_mse)

(6, 1.3, 1.6, 1.55) 11772.753878945423
(6, 1.3, 1.6, 1.55) 280597860.7426168


In [ ]:
'''Run best random forest hyperparam on full dataset'''
test_gdf.drop(columns=["opening_year"], inplace=True, errors='ignore')
train_df, test_df = preprocess_train_test(train_gdf, test_gdf)

x_train, y_train = train_df.drop(columns=["resale_price"], errors='ignore'), train_df["resale_price"]
features = pd.concat([x_train, test_df])

# features_tmp = features
features_tmp = features.iloc[:, :60]
# features_tmp = features.drop(columns=['distance_to_mrt_bins_price', 'mrt_lrt_links', 'population_bins_price', 'adult_children_ratio_bins_price', 'mrt_interchange_flag', 'mrt_interchange_count', 'mrt_type_price'], errors='ignore')
# features_tmp = features.drop(columns=['distance_to_mrt_bins_price', 'mrt_lrt_links', 'population_bins_price'], errors='ignore')

x_train = features_tmp.iloc[:len(train_df), :]
x_test = features_tmp.iloc[len(train_df):, :]


regressor = xgb.XGBRegressor(n_estimators=1800, max_depth=5, min_child_weight=1.3, reg_alpha=1.6, reg_lambda=1.5, n_jobs=3).fit(x_train, y_train)
# regressor = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=6, 
#                              min_child_weight=1.5, n_estimators=500,
#                              reg_alpha=1, reg_lambda=0.8571,
#                              subsample=0.5213, nthread = -1).fit(x_train, y_train)

y_train_predict = regressor.predict(x_train)
# y_valid_predict = regressor.predict(x_valid)

print(mean_absolute_error(y_train, y_train_predict))
# mae_valid.append(mean_absolute_error(y_valid, y_valid_predict))
print(mean_squared_error(y_train, y_train_predict))
# mse_valid.append(mean_squared_error(y_valid, y_valid_predict))

In [24]:
y_test_predict = regressor.predict(x_test)


In [26]:
submission = pd.DataFrame({
        "Predicted": y_test_predict
    })

submission.to_csv("PredictedPrice.csv", index=True, index_label="Id")


## Lightgbm

In [14]:
num_leaves = [4, 6]
max_bins = [100, 200]
bagging_fractions = [0.7, 0.8] 
bagging_freqs = [3, 10]

# num_leaves = [4]
# max_bins = [100, 400]
# bagging_fractions = [0.75] 
# bagging_freqs = [3]

scores_lightgbm = {}
for num_leave in num_leaves:
    for max_bin in max_bins:
        for bagging_fraction in bagging_fractions:
            for bagging_freq in bagging_freqs:
                print('start training with num_leave: {}, max_bin: {}, bagging_fraction: {}, bagging_freq: {}'.format(num_leave, max_bin, max_bin, bagging_fraction))
                mae_test, mae_valid, mse_test, mse_valid = [], [], [], []
                split = KFold(n_splits=5, shuffle=True).split(train_gdf)
                for idx, (train_idx, validate_idx) in enumerate(split):
                    print('fold {}'.format(idx + 1))
                    train_df, validate_df = preprocess_train_test(train_gdf.iloc[list(train_idx)], train_gdf.iloc[list(validate_idx)])

                    x_train, y_train = train_df.drop(columns=["resale_price"], errors='ignore'), train_df["resale_price"]
                    x_valid, y_valid = validate_df.drop(columns=["resale_price"], errors='ignore'), validate_df["resale_price"]

                    features = pd.concat([x_train, x_valid]).reset_index(drop=True)
                    overfit = []
                    for i in features.columns:
                        counts = features[i].value_counts()
                        zeros = counts.iloc[0]
                        if zeros / len(features) * 100 > 99.94:
                            overfit.append(i)

                    overfit = list(overfit)
                    final_features = features.drop(overfit, axis=1)

                    x_train = final_features.iloc[:len(x_train), :]
                    x_valid = final_features.iloc[len(x_train):, :]

                    regressor = LGBMRegressor(objective='regression', 
                                                n_estimators=300,
                                                num_leaves=num_leave,
                                                max_bin=max_bin, 
                                                learning_rate=0.01, 
                                                bagging_fraction=bagging_fraction,
                                                bagging_freq=bagging_freq, 
                                                bagging_seed=7,
                                                feature_fraction=0.2,
                                                feature_fraction_seed=7).fit(x_train, y_train)                    
                    y_train_predict = regressor.predict(x_train)
                    y_valid_predict = regressor.predict(x_valid)

                    mae_test.append(mean_absolute_error(y_train, y_train_predict))
                    mae_valid.append(mean_absolute_error(y_valid, y_valid_predict))
                    mse_test.append(mean_squared_error(y_train, y_train_predict))
                    mse_valid.append(mean_squared_error(y_valid, y_valid_predict))

                print(mae_test, mae_valid, mse_test, mse_valid)
                scores_lightgbm[(max_depth, min_child_weight, reg_alpha, reg_lambda)] = (mae_test, mae_valid, mse_test, mse_valid)

start training with num_leave: 4, max_bin: 100, bagging_fraction: 100, bagging_freq: 0.7
fold 1
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
fold 2
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
fold 3
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignor

In [15]:
'''Run best random forest hyperparam on full dataset'''
test_gdf.drop(columns=["opening_year"], inplace=True, errors='ignore')
train_df, test_df = preprocess_train_test(train_gdf, test_gdf)

x_train, y_train = train_df.drop(columns=["resale_price"], errors='ignore'), train_df["resale_price"]

# x_valid, y_valid = validate_df.drop(columns=drop_columns, errors='ignore'), validate_df["resale_price"]

regressor = LGBMRegressor(objective='regression', 
                            n_estimators=500,
                            num_leaves=num_leave,
                            max_bin=max_bin, 
                            learning_rate=0.01, 
                            bagging_fraction=bagging_fraction,
                            bagging_freq=bagging_freq, 
                            bagging_seed=7,
                            feature_fraction=0.2,
                            feature_fraction_seed=7).fit(x_train, y_train)
y_train_predict = regressor.predict(x_train)
# y_valid_predict = regressor.predict(x_valid)

print(mean_absolute_error(y_train, y_train_predict))
# mae_valid.append(mean_absolute_error(y_valid, y_valid_predict))
print(mean_squared_error(y_train, y_train_predict))
# mse_valid.append(mean_squared_error(y_valid, y_valid_predict))

In [16]:
min_mae_key, min_mae = None, 100000
min_mse_key, min_mse = None, 10000000000000
for key, (_, mae_valid, _, mse_valid) in scores_lightgbm.items():
    if np.mean(mae_valid) < min_mae:
        min_mae_key = key
        min_mae = np.mean(mae_valid)
    if np.mean(mse_valid) < min_mse:
        min_mse_key = key
        min_mse = np.mean(mse_valid)
print(min_mae_key, min_mae)
print(min_mse_key, min_mse)

(6, 2, 1, 1) 45387.7810192872
(6, 2, 1, 1) 3554484391.690834
